# Hillfort detection with LiDAR data
## Data management

## Table of contents

[Code](#code)

1. [**Defined functions**](#defined-functions)
2. [**Data gathering**](#data-gathering)
3. [**Data preprocessing**](#data-preprocessing)
4. [**Data augmentation**](#data-augmentation)
5. [**Data saving**](#data-saving)

[End](#end)

## Code

### Defined functions

In [1]:
# Imports
# import os
# import re
# import csv
# import typing
# import itertools
# import json
# import logging
# import warnings
# import evaluate
# import types
import pandas as pd
import sklearn as sk
import numpy as np
# import torch
# import math
import shapely
import matplotlib.pyplot as plt
import laspy # Reading LAS file format
from tqdm import tqdm # Loading bars
import geopandas as gpd
import lib.download_maps

In [2]:
# Constants
...

In [ ]:
def gather_data():
    lib.download_maps.process_csv('../data/linnamagede_ruudunumbrid.csv')

In [4]:
gather_data()

137it [00:14,  9.53it/s]
2024-11-18 19:29:38,794 - INFO - Total files: 153


In [99]:
# Load area points from the text file and create a polygon
def load_area_polygon(file_path):
    coordinates = []
    with open(file_path, 'r') as f:
        for line in f:
            if len(line.split()) == 3:
                _, x, y = line.strip().split()
                coordinates.append((float(y), float(x)))  # Swap X and Y for the polygon
    return shapely.Polygon(coordinates)

In [100]:
# Modify LAS points' classification inside the polygon
def classify_points_in_area(las_file_path, polygon, new_classification, output_path):
    # Load the LAS file
    las = laspy.read(las_file_path)
    
    # Extract LAS point X, Y (swapped) coordinates as tuples
    points = np.column_stack((las.y, las.x))  # Swap X and Y
    
    # Create a mask for points inside the polygon
    mask = np.array([polygon.contains(shapely.Point(pt)) for pt in points])

    # Apply the new classification to points within the polygon
    las.classification[mask] = new_classification
    
    # Write modified points to a new LAS file
    las.write(output_path)

In [101]:
# Rasterize a shapely polygon for overlay
def rasterize_polygon(polygon, x_min, x_max, y_min, y_max, resolution):
    grid = np.zeros((resolution, resolution), dtype=np.uint8)
    x_range = np.linspace(x_min, x_max, resolution)
    y_range = np.linspace(y_min, y_max, resolution)
    
    for i, y in enumerate(y_range):
        for j, x in enumerate(x_range):
            if polygon.contains(shapely.Point(x, y)):
                grid[i, j] = 1  # Mask for polygon
    return grid

In [121]:
# Function to blend mask into an image with transparency
def apply_transparency(image, mask, mask_color, alpha):
    # Create a copy of the original image to avoid overwriting
    blended_image = image.copy()
    
    # Get indices where the mask is active
    mask_indices = mask == 1  # Binary mask where 1 indicates the mask area
    
    # Blend mask color with the original image in the masked region
    for channel in range(3):  # Loop over RGB channels
        blended_image[..., channel][mask_indices] = (
            (1 - alpha) * blended_image[..., channel][mask_indices] + alpha * mask_color[channel]
        )
    
    return blended_image

In [102]:
def preprocess_data():
    mask_hillforts()

### Data gathering

*Kirjeldada, kuidas andmed koguti, mis takistused ja neile lahendused olid, kellega kokku puutusime andmete kogumisel*

In [ ]:
# Andmete allalaadimine

### Data preprocessing

*Kirjeldada lühidalt andmetöötluse protsesse: tunnuste valimine, filtreerimine, hulkadesse (treening-, val-?, test-) jagamine, augmentation andmete juurde genereerimiseks*

## End